In [ ]:
!az Login

In [ ]:
import azureml.core
from azureml.core import Workspace

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

In [ ]:
from azureml.core.image import Image
image = Image(ws, name="myimage", id=None, tags=None, properties=None, version=None)

container_reg = ws.get_details()["containerRegistry"]
resource_group_name = ws.resource_group
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id

print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))

In [ ]:
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

In [ ]:
iot_hub_name="iot_hub_name" # iot_hub_name入力
iot_device_id="iot_device_id" # iot_device_id入力
module_name = "aml-module" # AMLのモジュール名入力

file = open('iot-deployment-template.json')

contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# # Add extension
# !az extension add --name azure-iot
# !az extension add --name azure-cli-iot-ext

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json